<a href="https://colab.research.google.com/github/AshishRaj04/nano-gpt/blob/main/nano_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
path = "/content/drive/MyDrive/Projects/Transformers/THE PROSE WORKS OF WILLIAM WORDSWORTH.txt"

In [3]:
with open(path , "r" , encoding = "utf-8") as f:
  text = f.read()

In [4]:
print("length of dataset in character ", len(text))

length of dataset in character  2744711


In [5]:
print(text[ : 500])

TO THE QUEEN.

MADAM,

I have the honour to place in your Majesty's hands the hitherto
uncollected and unpublished Prose Works of

WILLIAM WORDSWORTH

--name sufficient in its simpleness to give lustre to any page.

Having been requested thus to collect and edit his Prose Writings by
those who hold his MSS. and are his nearest representatives, one little
discovery or recovery among these MSS. suggested your Majesty as the one
among all others to whom the illustrious Author would have chosen to
d


In [6]:
import re

clean_text = re.sub(r"[^a-zA-Z0-9\s.,;:'\"?!\-]", "", text)

In [7]:
chars = sorted(list(set(clean_text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !"',-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
73


In [8]:
str_to_idx = {ch:i for i,ch in enumerate(chars)} #  { 47: 'a', 48: 'b', 49: 'c', }

idx_to_str = {i:ch for i,ch in enumerate(chars)} # { 'a': 47, 'b': 48, 'c': 49, }

In [9]:
encode = lambda s : [str_to_idx[c] for c in s]
decode = lambda l : "".join([idx_to_str[i] for i in l])

In [10]:
print(encode("ashish raj prasad"))
print(decode(encode("ashish raj prasad")))

[47, 65, 54, 55, 65, 54, 1, 64, 47, 56, 1, 62, 64, 47, 65, 47, 50]
ashish raj prasad


In [11]:
data = tf.Variable(encode(clean_text) , dtype = tf.int64)

In [12]:
n = int(0.9 * len(data.numpy()))
train_data = data[ : n]
val_data = data[n : ]

In [13]:
context_length = 8
train_data[ : context_length+1]

<tf.Tensor: shape=(9,), dtype=int64, numpy=array([40, 35,  1, 40, 28, 25,  1, 37, 41])>

In [14]:
x = train_data[ : context_length]
y = train_data[1 : context_length+1]

for t in range(context_length):
  context = x[ : t+1]
  target = y[t]
  print(f"when input is {context} the target is {target}")

when input is [40] the target is 35
when input is [40 35] the target is 1
when input is [40 35  1] the target is 40
when input is [40 35  1 40] the target is 28
when input is [40 35  1 40 28] the target is 25
when input is [40 35  1 40 28 25] the target is 1
when input is [40 35  1 40 28 25  1] the target is 37
when input is [40 35  1 40 28 25  1 37] the target is 41


In [15]:
seed = tf.random.set_seed(424242424224)
batch_size = 4
context_len = 8

def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = tf.cast(tf.random.uniform( shape = (batch_size,), minval = 0, maxval = len(data)-context_len , seed = seed) , dtype=tf.int64) # 4 random int btw 0 and len(train_data)
  x = tf.stack([data[i : i+context_len] for i in ix])
  y = tf.stack([data[i+1 : i+context_len+1] for i in ix])
  return x , y

xb , yb = get_batch("train")

print("Inputs:")
print(xb.shape)
print(xb)
print("Targets:")
print(yb.shape)
print(yb)

# print("-----")

# for b in range(batch_size):
#   for t in range(context_len):
#     context = xb[b, :t+1]
#     target = yb[b,t]
#     print(f"when input is {context.numpy()} the target is {target}")


Inputs:
(4, 8)
tf.Tensor(
[[ 1 47  1 48 64 55 53 54]
 [65 55 66 55 61 60  5  1]
 [60  1 61 52  1 51 70 62]
 [50  1 47 69 47 71  7  1]], shape=(4, 8), dtype=int64)
Targets:
(4, 8)
tf.Tensor(
[[47  1 48 64 55 53 54 66]
 [55 66 55 61 60  5  1 66]
 [ 1 61 52  1 51 70 62 61]
 [ 1 47 69 47 71  7  1 40]], shape=(4, 8), dtype=int64)


Finding base line performance on bigram language model

In [16]:
class BigramLanguageModel(tf.keras.Model):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, vocab_size)  # 73x73 table

    def call(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B, T, C)
        if targets is None:
            loss = None
        else:
            logits_flat = tf.reshape(logits, (-1, logits.shape[2]))  # (B*T, C)
            targets_flat = tf.reshape(targets, (-1,))  # (B*T,)
            loss = tf.keras.losses.sparse_categorical_crossentropy(
                targets_flat, logits_flat, from_logits=True
            )
            loss = tf.reduce_mean(loss)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.call(idx)
            logits = logits[:, -1, :]  # (B, C)
            # Note: no softmax here, directly sample from logits
            idx_next = tf.random.categorical(logits, num_samples=1)  # (B, 1)
            idx = tf.concat([idx, idx_next], axis=1)  # (B, T+1)
        return idx


In [17]:
m = BigramLanguageModel(vocab_size)
out , loss = m(xb , yb)
print("out shape---->",out.shape)
print("loss---->",loss.numpy().item())

out shape----> (4, 8, 73)
loss----> 4.297455787658691


In [18]:
idx = tf.zeros((1,1) , dtype = tf.int64)
print(decode(m.generate(idx , max_new_tokens=1000)[0].numpy().tolist()))


OyqizF.Fdq :s,wZ-O8,?GZqAE7n"ZD9ZCa,-6r.dRBixLq6ef!rQH;sIkiO,IfBL
Ikz;DXRq4Ss3eFMH.cMQjeQrBA7i1O?G:0j4ax6zf5oH;NQWlXcOLPmSgeslX kynbS,"Ue?E69XfI4"L8eYMqmvRRljBl0ZQGGyY?WTYT
HdBS6j6KR. LkBAcB;a'MbC86wqKdsd"7.Wj?z2DHm0?ZNffnUitLk"2AF
.zJeXdDArz12GqTJ 7;LcpaL
Di"!W25BW?oyyJP1s25Y":YYZgAwa6Ri4FFQA:bnT7J?:'OoMcQ.d9yD.7WAaKFhYysRpk ,,d S'HoL-oE,l3.Q54L:NE ;BHnEhjsdG2rLIy66sNZAre.e5bFR9C3b37S'uA3gNo0xLutaj lhGNXK,nR
FARTmgrfwIus:?sPvSE?Iuty2MohMeHxb'hB-."o2ub5XoS7Hid PGTrHdB!rRuBgETZmLxik,TIwA 6GLr
:1"T52ZAUgi.
mT.G
adp1u0tNG1,HLnBcgD5:1sz:vWUg7HqK2?gtpAJ;ZFPwozH'AnIImMPV4YIe4,QEB2HPJ: B-;lS2Ex,uORU6Q5Nx:sp-nRb2769?8lI"I",CjNQ7ia9VrN1sgPR:YQG7"R,T9I
qZn,9D8fFTmszW;W7z9U77Bs!?GRvu.Mddr4eg
QnJo-Fwv6.fJrB: tG; jv KOhTnxCrcgY
3A5sA'vBKbDDVxryT0qEe;BZ24XmL2th.PHfc1ndVS;LQ8M."vqcIaOir0RcJaQJ"m7VfrHXV!RE0;b2AZsZY3aAz'x2y ,99mE!gadI?Sz'JW?othzMsqeu:UpcPp3l
O
H!L"i?pY-?Cgkh
Eo42WWs6e?dTh8DIWrjibBfq YR1Hs.S'?rBMioeC
,bq,t:zwQ,KpgK-1u.D2MYRcch0PEm:e'81
v0LXVrK!oim0Pt,2VW;,axhVeG5pMXYlRRA
I1TQIEslsCVguh

In [19]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

In [20]:
for i in range(10000):
  with tf.GradientTape() as Tape:
    xb , yb = get_batch("train")
    logits , loss = m(xb , yb)
  grads = Tape.gradient(loss , m.trainable_variables)
  optimizer.apply_gradients(zip(grads , m.trainable_variables))

print(loss.numpy().item())


2.370056629180908


In [21]:
print(loss.numpy().item())

2.370056629180908


In [22]:
print(decode(m.generate(idx , max_new_tokens=1000)[0].numpy().tolist()))


es ind tlvXD; n t.
ie atemaiblflourtheplld, haphedig wnt,  Rye or--ICORELeat pth aing wlley:by digse:  s, or abin Drecefin in Wismon wh th 5dule tund, e thrs
ghe olar w.PESiothearses FR. o. 5s tha ablee d coff pot'athad, wesac. urind h ourend in tsa  pe us wang ht m tilin s.

3'Hbrerong gs rr  orer onde She ticennv178DJherrsoviseare t pro
 theq;QOd
Maremede sre scouchiaf wrmutionde Ves spl RZqW.
ilof
XVwa t TIseokAe Ms p ant owhes; ilithe EOcatan ofrked tishain, tousthenghepo bss wet ff  ano me iound  aco sind orifanceas ture sor
B ude ened d bs  IXNDXJ r pped iofr----TDDal a:X
ce  JI ond is. Cq1 benge anviof chiut, presthawie aty ws mpon  t, owingomprdonagens hthenthld t ibemm thnd wa gurnce cte ced ul ining this-"C-30?wf Hosunoste imewrardinqure tuitouse ' an wicof
oe
an. se tren pe,,  f
 w Seen agt DJOjomere?Th ts.H-LLf tthan fe tofin ateppe malveas; waf guregex, T8?vely tht temutouis s Eur wint the her perr, o l sly
aswhe  ade rot whe ghele pancha
ABionol ang o
uck Czr; dee o chen

Self Attention

In [23]:
B , T , C = 4 , 8 , 2
x = tf.random.uniform(shape=(B , T , C) , minval = 0 , maxval = 10)
x.shape

TensorShape([4, 8, 2])

In [24]:
wei = tf.keras.ops.tril(tf.ones((T , T)))
wei = wei / tf.reduce_sum(wei , axis = 1 , keepdims=True)
wei.shape

TensorShape([8, 8])



```
> - wei @ x --- > [T , T] @ [B , T , C]
> - tf will add a batch dim [B] at the start of wei . now wei --> [B , T , T]
> - wei @ x --- > [B , T , T] @ [B , T , C]
> - for every batch matmul applies . [T , T] @ [T , C] --> [T , C]
> - result will be [B , T , C]
```







**Scenario**:

> - a: A lower triangular matrix of ones with dimensions [n, n]. It's normalized along each row (axis 1) such that the elements in each row sum to 1.
This normalization makes it a weighted averaging matrix.
> - b: A matrix with dimensions [n, m] that is compatible for multiplication with a (e.g., [3, 2]).
> - c: The result of the matrix multiplication a @ b.

**Result**:

Each row of c represents a weighted average of the current and preceding rows of b. The weights are determined by the elements in the corresponding row of a.

**Example**:


```
a = tf.Variable([[1. , 0. , 0. ],
     [0.5, 0.5, 0. ],
     [0.3, 0.3, 0.3]] )

b = tf.Variable([[4., 2.],
     [6., 2.],
     [1., 3.]])

c = a @ b

c
```
**Output**:


```
<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[4.       , 2.       ],
       [5.       , 2.       ],
       [3.3      , 2.1000001]], dtype=float32)>
```




In [25]:
xbow = tf.matmul(wei , x)
xbow.shape

TensorShape([4, 8, 2])

Another way of doing the same

In [26]:
tril =  tf.keras.ops.tril(tf.Variable(tf.ones((T,T))))
wei = tf.Variable(tf.zeros((T,T)))
mask = (tril == 0)
wei = tf.where(mask, float('-inf'), wei)
wei = tf.nn.softmax(wei , axis = -1)
xbow2 = wei @ x

In [27]:
xbow2.shape

TensorShape([4, 8, 2])

Decoder only transformer

In [ ]:
n_embd = 32

In [ ]:
class BigramLanguageModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, n_embd)  # 73x32 table
        self.position_embedding_table = tf.keras.layers.Embedding(context_length, n_embd)  # 8x32 table
        self.lm_head = tf.keras.layers.Linear(n_embd, vocab_size)  # 32x73

    def call(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)  # (B, T, C)
        pos_emb = self.position_embedding_table(tf.range(T))  # (T, C)
        x = tok_emb + pos_emb  # (B, T, C)
        logits = self.lm_head(tok_emb)  # (B, T, vocab_size)
        if targets is None:
            loss = None
        else:
            logits_flat = tf.reshape(logits, (-1, logits.shape[2]))  # (B*T, C)
            targets_flat = tf.reshape(targets, (-1,))  # (B*T,)
            loss = tf.keras.losses.sparse_categorical_crossentropy(
                targets_flat, logits_flat, from_logits=True
            )
            loss = tf.reduce_mean(loss)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.call(idx)
            logits = logits[:, -1, :]  # (B, C)
            # Note: no softmax here, directly sample from logits
            idx_next = tf.random.categorical(logits, num_samples=1)  # (B, 1)
            idx = tf.concat([idx, idx_next], axis=1)  # (B, T+1)
        return idx
